<font size=6>

<b>Curso de Análisis de Datos con Python</b>
</font>

<font size=4>
    
Curso de formación interna, CIEMAT. <br/>
Madrid, Junio de 2023

Antonio Delgado Peris (Cristina Labajo Villaverde)
</font>

https://github.com/andelpe/curso-python-analisis-datos

<br/>

# Tema 8 - Manipulación de datos avanzada con Pandas

## Objetivos

- Profundizar en el análisis y manipulación de datos con la librería Pandas.

- Ver maneras más sofisticadas de seleccionar y filtrar los datos de un DataFrame.

- Introducir el concepto de multi-índice.

- Conocer nuevas formas de transformación de un DataFrame.

- Mejorar nuestras capacidades de gestión de datos con referencias temporales.

## Vistas y copias

En el tema 4 vimos que algunas operaciones de NumPy podían devolver objetos _vista_ a un ndarray existente, en lugar de un nuevo objeto (copia). Las reglas eran bastante sencillas: las operaciones de _slice_ con índices sencillos retornan una vista, y las de índice complejo (con más de un elemento, o por máscara) retornan copias.

En Pandas, sin embargo, es muy difícil saber si una operación de selección/indexado de un DataFrame va a devolver una vista o una copia (depende de muchas cosas, y no está bien definido).

Eso nos lleva a sugerir que, en general, no asumamos ni una cosa ni la otra, y si queremos estar seguro que una subselección de un DataFrame es una copia, usemos explícitamente el método `copy`.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame({'a':[1, 2], 'b':[3, 4]})
df

In [ ]:
my_slice = df.iloc[1:,]
my_slice

In [ ]:
my_slice.iloc[0, 1] = 99
my_slice

In [ ]:
df

In [ ]:
my_slice.iloc[0, 0] = 10.4
display(my_slice)
display(df)

Como vemos, en un caso un cambio en `my_slice` produce un cambio en `df`, y en el otro no.

Como también hemos podido apreciar en los ejemplos anteriores, los desarrolladores de Pandas son conscientes de este problema, han establecido un mecanismo de alerta. Siempre que se detecta que se quiere modificar una subselección de un DataFrame se genera un `SettingWithCopy Warning`.

Si usamos `copy`, al menos estaremos seguros de lo que estamos haciendo.

In [ ]:
my_slice = df.iloc[1:,].copy()
my_slice

In [ ]:
my_slice.iloc[0, 0] = 1000
my_slice

In [ ]:
df

In [ ]:
my_slice.iloc[0, 1] = 10.4
display(my_slice)
display(df)

<br/>

Debemos resaltar que en algunas ocasiones este problema se manifiesta en operaciones más compactas en que lo queremos precisamente es modificar el DataFrame original, pero no ocurre (porque estamos modificando una copia).

Esto sucede cuando encadenamos varias operaciones de indexado en una asignación (_index chaining_). Por ejemplo:

In [ ]:
df = pd.DataFrame({'a':[1, 2], 'b':[3, 4]})
display(df)
df[df['b'] > 3]

In [ ]:
df[df['b'] > 3]['a'] = 100
display(df)

In [ ]:
(df[df['b'] > 3])['a'] = 100
display(df)

En este caso, la solución pasa por usar un único indexado en la asignación, en lugar de encadenar varios:

In [ ]:
df.loc[df['b'] > 3, 'a'] = 100
df

**En resumen:**

- Si queremos modificar el DataFrame original debemos utilizar un indexado único (como en el último ejemplo).
- Si lo queremos es tener una copia independiente del original (y asegurar que no se modifica este), lo más conveniente es utilizar el método `copy` explícitamente.
- Si nos da igual, podemos simplemente ignorar el Warning de Python

## Multi-índices
Los multi-índices (o índices jerárquicos) permiten tratar datos como si tuvieran un número arbitrario de dimensiones (en lugar de solo 2). Se puede considerar que un multi-índice es una serie ordenada de tuplas, donde cada tupla es única.

Las operaciones con índice jerárquico están sujetas a las mismas reglas de alineado que con índice simple.

Por ejemplo, podemos crear un multi-índice explícitamente de una serie de tuplas.

In [ ]:
# From tuples
tuples = [("España", "Madrid"), ("España", "Valencia"), ("España", "Sevilla"),
          ("Francia", "Paris"), ("Francia", "Lyon")]

ciudades = pd.MultiIndex.from_tuples(tuples, names=["Pais", "Ciudad"])

s = pd.Series([100, 125, 90, 160, 50], index=ciudades)
s

<br/>

O generando una combinación de elementos, todos con todos (i.e.: el índice es completo en todos los niveles).

In [ ]:
# From cross product 
iterables = [["1º", "2º"], ["A", "B", "C"]]
index = pd.MultiIndex.from_product(iterables, names=["Curso", "Clase"])

cols = ('c1', 'c2', 'c3')
cole = pd.DataFrame(np.arange(18).reshape(6,3), index=index, columns=cols)
cole

<br/>

Los índices jerárquicos también pueden usarse en el otro eje (columnas), aunque es menos común.

In [ ]:
# Multi-index on the columns axis
df = pd.DataFrame(np.arange(18).reshape(3,6), columns=index, index=cols)
df

In [ ]:
# Volvemos a cambiar los ejes 
cole = df.T
cole

### Niveles
Observando el índice, se aprecia cómo está construido por una serie de tuplas únicas. La posición en estas tuplas define una jerárquía.

In [ ]:
index

In [ ]:
index.to_list()

<br/>

El multi-índice se estructura en niveles, que pueden ser accedidos, e incluso manipulados.

In [ ]:
index.levels

In [ ]:
index.levels[1]

In [ ]:
df2 = cole.reorder_levels(['Clase', 'Curso'])
df2

Podemos ordenar en base al nuevo índice, para ver los datos más ordenados.

In [ ]:
df2.sort_index()

<br/>

También existen métodos para promocionar columnas a índice o al revés. Se debe resaltar que las búsquedas en un índice son más rápidas que en una columna normal.

In [ ]:
cole

In [ ]:
cole.reset_index('Clase', inplace=True)
cole

In [ ]:
cole.set_index('Clase', append=True, inplace=True)
cole

<p/>

NOTA: Se se usa el argumento `drop=True`, el índice eliminado, no se promociona a columna, sino que se descarta.

In [ ]:
temp = cole.reset_index(drop=True)
temp

Esto puede ser útil, por ejemplo, para descartar un índice poco útil, y generar uno numérico secuencial.

In [ ]:
temp = temp.filter(items=[2,4], axis=0)
temp

In [ ]:
temp.reset_index(drop=True)

### Indexado y selección con multi-índices

Los mismos métodos que vimos con índices no-jerárquicos pueden ser usados con multi-índices, aunque algunas operaciones (_slice_) son algo más complejas.

Recordemos los principales métodos de indexado:

- Selección de una columna con `df[colName]` o `df.colName`.
- Selección de varias columnas con `df[[col1, col2...]]`.
- Selección de filas con `df[slice]`, usando posición o etiquetas.
- Selección de filas y columnas por etiqueta, usando `df.loc`.
- Selección de filas y columnas por posición, con el método `df.iloc`.
- Filtrado por máscara

In [ ]:
# Selección de columnas
# NOTA: Recordemos que para selección múltiple, usamos una lista
#       Las tuplas se interpretan como claves jerárquicas del índice
cole[['c1', 'c3']]

In [ ]:
# Slice en filas desde 1º-B e incluyendo todo 2º
cole[('1º', 'B'):'2º']

In [ ]:
# O usando posición del índice
cole[1:4]

In [ ]:
# .loc con etiquetas 
cole.loc["1º"]

In [ ]:
cole.loc[("1º", "B"), "c2"]

In [ ]:
# Slice en índice de 1ºB a 2ºB, para columnas de c1 a c2
cole.loc[('1º', 'B'):('2º', 'B'), 'c1':'c2']

Para realizar _slices_ en diferentes niveles, podemos usar un objeto `IndexSlice`.

Por ejemplo, para recuperar todos los grupos `B`, independientemente del curso.

In [ ]:
idx = pd.IndexSlice

cole.loc[idx[:, 'B'], ['c1', 'c3']]

Finalmente, también podemos filtrar _valores_ (no el índice) con una máscara booleana, a la manera habitual, o combinándolo con con el `Indexslice`.

In [ ]:
mask = cole['c2'] > 7
cole[mask]

In [ ]:
cole.loc[idx[mask, 'C'], 'c2']

### Otros métodos para selección de datos

Veamos algunos otro métodos para seleccionar elementos de un DataFrame (con índice sencillo o jerárquico).

El método `isin` permite filtrar por _valores_ que se encuentren en una lista dada, o, del mismo modo, por los contenidos de un índice.

In [ ]:
s = pd.Series({'b': 4, 'a': 9 , 'c': 10})
s[s.isin([2, 4, 6])]

In [ ]:
mask = cole.isin(range(3, 10))
cole[mask]

In [ ]:
grupos = [('1º', 'B'), ('2º', 'C')]
cole[cole.index.isin(grupos)]

El método `query` puede usarse para filtrar en base a condiciones, igual que se hace con máscaras booleanas, pero de manera más concisa. Por ejemplo, las dos líneas siguientes son equivalentes:

```python
df[(df['a'] < df['b']) & (df['b'] < df['c'])]

df.query('(a < b) & (b < c)')
```

Se pueden introducir condiciones en los valores (columnas) y también en los índices.

In [ ]:
cole.query('(c3 > 13) & (Clase=="B")')

<div style="background-color:powderblue;">

**EJERCICIO e8_1:** 

Cargar los datos de `data/WordsByCharacter.csv` en un DataFrame en el que todas las columnas forman parte de un multiíndice, excepto `Words` (para ello podemos usar `index_col` con `read_csv`, o usar el método `set_index` a posteriori).
    
A partir de este DataFrame, contestar a las siguientes preguntas:
    
- ¿Qué personajes hablan en el primer capítulo de _The Fellowship Of The Ring_? ¿Cuántas palabras cada uno?
- ¿Quiénes son los 3 primeros elfos en hablar en esa misma película? (Avanzado: ¿Y los primeros 5?)
- ¿Cuánto hablan Gandalf y Saruman en cada capítulo de _The Two Towers_?

## Operaciones de agrupación
Una tarea habitual a realizar con nuestros datos es agruparlos en función de determinados criterios, y realizar operaciones con los grupos resultantes.

### groupby 
La función `groupby` permite agrupar los elementos de un DataFrame en función de criterios en los valores de columnas o índice.

In [ ]:
cole

In [ ]:
# Agrupamos según uno de los índices
groups = cole.groupby('Clase')
print(groups) 

In [ ]:
groups.get_group('A')

In [ ]:
for name, group in groups: 
    print('NAME:', name)
    print()
    print('GROUP:\n', group, '\n---------------------\n')

Podemos aplicar una función de agregación para ver el resultado de aplicarlo sobre cada grupo.

In [ ]:
groups.sum()

In [ ]:
# Simplificamos ahora los valores para ilustrar la agrupación por valores en columnas
df2 = cole.copy()
df2['c1'] = [0, 1] * 3
df2['c2'] = [0, 1, 1] * 2
df2

In [ ]:
# Agrupamos considerando valores de una columna o varias
groups = df2.groupby(['c1', 'c2'])
groups.indices

Podemos acceder a un grupo determinado con `get_group`.

In [ ]:
groups.get_group((0,1))

In [ ]:
groups.count()

Como argumento a `groupby` también podemos usar una función, que se aplica a las etiquetas a agrupar, un diccionario con agrupamiento explícito de etiquetas, y otros.

Por ejemplo, agrupar los cursos 'B' por un lado, y el resto en otro grupo.

In [ ]:
def myfunc(label):
    if label[1] == 'B': return 'G2'
    else:               return 'G1'

for name, group in cole.groupby(myfunc):
    print(name, '->', group.index.to_list())

In [ ]:
cole.groupby(myfunc).mean()

<br/>

En estos casos, también podemos agrupar columnas, en lugar de filas. El siguiente ejemplo agrupa por un lado las columnas `c1` y `c2`, y por otro `c3`.

In [ ]:
mapping = dict(c1='G1', c2='G1', c3='G2')
cole.groupby(mapping, axis=1).mean()

<br/>

Incluso podríamos agrupar por una combinación de filas y columnas (Pandas inferirá a cuál nos referimos por las etiquetas existentes en el DataFrame).

In [ ]:
df3 = pd.DataFrame([[0, 0, 0], [1, 1, 1], [0, 1, 2]], index=pd.Index(('a', 'a', 'b'), name='Idx'), columns=('c1', 'c2', 'c3'))
df3

In [ ]:
# Agrupamos por índice 
df3.groupby('Idx').count()

In [ ]:
# Agrupamos por las diferentes combinaciones de índice y columna c2
df3.groupby(['Idx', 'c2']).count()

### Agregación y transformación
Igual que se pueden aplicar funciones de agregación (reducción) sobre un DataFrame, se puede hacer sobre los grupos resultantes del uso de `groupby`. Para ello, usamos la función `aggregation` (o `agg`).

In [ ]:
cole

In [ ]:
cole.groupby('Curso').agg(np.max)

Si no queremos reducir la dimensionalidad sino solo modificar los elementos del DataFrame, podemos usar `transform`. La función argumento se aplicará sobre cada grupo resultado de `groupby`.

En el siguiente ejemplo, a cada clase queremos fijar en las columnas los valores máximo para el curso en el que está (de cualquier clase, A, B o C).

In [ ]:
cole.groupby('Curso').transform(lambda x: max(x))

In [ ]:
cole.groupby('Curso').agg(lambda x: max(x))

### Filtrado
La función `filter` aplicada a los resultados de una agrupación, los filtra, de modo que solo aquellos que satisfagan cierta condición sean devueltos. El resultado es un único DataFrame filtrado (no una serie de agrupaciones).

NOTA: No confundir con la función `df.filter` que permite filtrar los índices/columnas que queremos seleccionar (se aplica a las etiquetas, no a los valores).

La condición se expresa como una función arbitraria que se aplica a cada grupo y devuelve un boolean para cada caso.

In [ ]:
cole

In [ ]:
# Elegimos las clases (A, B o C) cuyo primer curso supere cierto umbral en c1
# Vemos que el resultado excluye completamente el grupo A (por 1ºA, aunque 2ºA sí lo cumpla)

cole.groupby('Clase').filter(lambda x: x.loc['1º']['c1'] >= 1)

<div style="background-color:powderblue;">

**EJERCICIO e8_2:** 

Siguiendo con el DataFrame del ejercicio anterior (`WordsByCharacter.csv`), contestar a las siguientes preguntas:
    
- ¿Cuánto hablan Gandalf y Saruman en total en _The Two Towers_?
    
- ¿Qué tres hobbits hablan más en el conjunto de las tres películas?
    
- ¿Qué tres hobbits hablan más en cada una de las tres películas?    

- ¿Cuál es el personaje que menos habla, pero que habla en más de 10 capítulos? (Nota: usar `filter`) 

## Unión de DataFrames

### Concat
Podemos usar la función `pd.concat` para unir una serie de DataFrames en uno nuevo resultante. Por defecto, se concatenan las filas.

En el proceso se hace una copia de los datos originales.

In [ ]:
dfA = pd.DataFrame(
    {
        "a": ["Aa1", "Aa2", "Aa3"],
        "b": ["Ab1", "Ab2", "Ab3"],
        "c": ["Ac1", "Ac2", "Ac3"],
    },
    index=[0, 1, 2],
)

dfB = pd.DataFrame(
    {
        "a": ["Ba1", "Ba2", "Ba3"],
        "b": ["Bb1", "Bb2", "Bb3"],
        "c": ["Bc1", "Bc2", "Bc3"],        
    },
    index=[10, 11, 12],
)

In [ ]:
dfA

In [ ]:
dfB

In [ ]:
pd.concat([dfA, dfB])

<br/>

El resultado de `concat` realizará, como siempre, una alineación de etiquetas en índice o columnas (el eje en el que no concatenemos).

In [ ]:
dfC = pd.DataFrame(
    {
        "a": ["Ca1", "Ca2", "Ca3"],
        "d": ["Cd1", "Cd2", "Cd3"],
        "c": ["Cc1", "Cc2", "Bc3"],        
    },
    index=[20, 21, 22],
)
dfC

In [ ]:
pd.concat([dfA, dfB, dfC])

<br/>

Podemos decidir no incluir en el resultado las columnas en las que no todos los DataFrames argumento tienen claves coincidentes, usando `join='inner'` (por defecto, se usa: `join='outer'`)

In [ ]:
pd.concat([dfA, dfB, dfC], join='inner')

<br/>

También podemos concatenar en la dimensión de columnas, con `axis=1`.

In [ ]:
dfD = pd.DataFrame(
    {
        "e": ["De1", "De2", "Df3"],
        "f": ["Df2", "Df2", "Df3"],
        "g": ["Dg3", "Dg2", "Dg3"],        
    },
    index=[0, 1, 2],
)
dfD

In [ ]:
pd.concat([dfA, dfD], axis=1)

<br/>

Por último, es interesante destacar que el método de la clase DataFrame `append` es equivalente a `pd.concat` (con sus principales argumentos con valores por defecto: `axis=0` y `join=outer`).

In [ ]:
dfA.append(dfB)

### Merge
La función `pd.merge` nos permite fusionar DataFrames, utilizando una lógica de álgebra de conjuntos, inspirada en SQL (bases de datos relacionales), y considerando ĺos índices (de las filas) o los valores de alguna(s) columna(s) como claves para relacionar los argumento.

Por defecto, `merge` considera como clave aquella(s) columna(s) que aparezca con el mismo nombre en los dos DataFrames usados como argumentos, y solo incluye en el resultado las filas con valores coincidentes para esa columna.

In [ ]:
dfA = pd.DataFrame(
    {
        "k": ["A", "B", "C"],
        "A": [1, 11, 111],
        "B": [2, 22, 222],
    },
    index=[0, 1, 2],
)
dfB = pd.DataFrame(
    {
        "k": ["A", "C", "D"],
        "C": [5, 55, 555],
        "D": [6, 66, 666],        
    },
    index=[0, 2, 3],
)

display(dfA)
display(dfB)

In [ ]:
pd.merge(dfA, dfB)

En general, usaremos `on` (o `left_on`/`right_on`) para especificar la columna a considerar como clave.

In [ ]:
pd.merge(dfA, dfB, on='k')

Además, `merge` ofrece diferentes posibilidades en la lógica de las filas a considerar según las claves que presenten en los argumentos:

- `inner` (defecto): solo las filas con claves coincidentes aparecen en el resultado.
- `outer`: todas las filas de los DataFrames argumento aparecen en el resultado.
- `left`: las filas del DataFrame _izquierdo_ (primer argumento) aparecen en el resultado.
- `right`: las filas del DataFrame _derecho_ (segundo argumento) aparecen en el resultado.

In [ ]:
pd.merge(dfA, dfB, how='outer')

In [ ]:
pd.merge(dfA, dfB, on='k', how='left')

In [ ]:
pd.merge(dfA, dfB, on='k', how='right')

<br/>

Podemos también hacer la fusión usando los índices como clave, en lugar de columnas.

In [ ]:
pd.merge(dfA, dfB, left_index=True, right_index=True)

Lo cual es más sencillo si el índice tiene un nombre.

In [ ]:
dfA.index.name = 'idx'
dfB.index.name = 'idx'

pd.merge(dfA, dfB, on='idx')

<br/>

La función `merge` es muy sofisticada y permite usar más de una columna como clave para la fusión, o nombres de columna diferentes para las claves los DataFrames fusionados, así como usar multi-índices. También permite fusionar más de 2 DataFrames a la vez.

Además, la función de DataFrame `join` ofrece parte de la funcionalidad de `pd.merge`. No lo vamos a ver aquí, aunque cabe resaltar que, al contrario de lo que ocurre con `merge`, con `join` se usan los índices como clave por defecto, aunque se puede indicar el uso de cierta(s) columna(s).

<div style="background-color:powderblue;">

**EJERCICIO e8_3:** 
    
Cargar los datos almacenados en los ficheros `school_2018_clean_Countries.pickle` (indicadores sobre escolarización, por país), y `development_2018_clean_Countries.pickle` (indicadores sobre desarollo, por país), y unirlos en una sola tabla usando `merge`. Observar el número de filas (países) en cada tabla.

Extra: usando `seaborn.regplot`, realizar un gráfico de regresión lineal entre alguna de las magnitudes incluidas en la tabla (p.ej. escuela secundaria y renta per capita).

## Pivotado de DataFrames
Pandas ofrece varias funciones para modicar la forma de nuestro DataFrame, esencialmente del formato _largo_ al _ancho_, y a la inversa. Recordando:

**Long-format**: cada variable es una columna, cada observación es una fila. 

**Wide-format**: se incluyen más columnas en la tabla, de modo que cada observación se define por las coordenadas de esa celda en cuanto a índice y columnas.

- De largo a ancho: `pivot`, `unstack`.
- De ancho a largo: `melt`, `wide_to_long`, `stack`.

### stack y unstack

In [ ]:
tuples = [('A', 'juan'), ('A', 'pedro'), ('B', 'maria')]
idx = pd.MultiIndex.from_tuples(tuples, names=["grupo", "nombre"])
df0 = pd.DataFrame(
    {
        'altura': [180, 175, 170],
        'peso': [80, 73, 65],
    },
    index=idx
)    
df0    

In [ ]:
# Con 'stack' lo hacemos más "larga" (perdemos columnas)
df1 = df0.stack()
pd.DataFrame(df1)

In [ ]:
# Con 'unstack' lo hacemos más "ancha" (ganamos columnas)
df1.unstack()

In [ ]:
# Todavía más
df0.unstack()

In [ ]:
df0.unstack()

In [ ]:
# Incluso hasta que todo sean columnas... en una serie
df0.unstack()

### pivot_tables
Las función `pivot_tables` nos permite promocionar valores de una tabla a índices o columnas, al tiempo que se usa alguna función de agregación para reducir los valores de estas diferentes celdas a una sola. De esta forma, podemos conseguir resúmenes de la información desde diferenes perspectivas. Esta funcionalidad es muy utilizada en los entornos de hoja de cálculo.

En Pandas podemos usar la función `pivot_tables` para ello.

- NOTA: también podríamos conseguir esta funcionalidad con `groupby`, pero seguramente `pivot_tables`
es más intuitivo.

In [ ]:
fname = 'data/long_format.csv'
gastos = pd.read_csv(fname, delim_whitespace=True)
gastos.head()

In [ ]:
# Lo siguiente fallará
gastos.pivot(index='nombre', columns='anyo', values='gasto')

<br/>

Con `pivot_table` podemos agregar el gasto por persona y año (debemos usar `aggfunc` porque la agregación por defecto calcula el valor medio).

In [ ]:
gastos.pivot_table(index='nombre', columns='anyo', values='gasto', aggfunc=np.sum)

Hagamos lo mismo con `groupby`

In [ ]:
gastos.groupby(['nombre', 'anyo'])['gasto'].sum().unstack()

<br/>

Cambiamos la perspectiva a ver las sumas por `concepto`, dando 0 como valor por defecto, y añadiendo sumas totales.

In [ ]:
gastos.pivot_table(index='anyo', columns='concepto', values='gasto', 
                aggfunc=np.sum, fill_value=0, margins=True)

In [ ]:
# Incluyamos ahora conceptos y nombres
gastos.pivot_table(index='anyo', columns=('concepto', 'nombre'), values='gasto', 
                aggfunc=np.sum, fill_value=0, margins=True)

<div style="background-color:powderblue;">

**EJERCICIO e8_4:** 

Contestar a las 3 primeras preguntas del ejercicio `e8_2` usando `pivot_table`, en lugar de `groupby`.

## Series temporales
Pandas ofrece un soporte muy potente para las unidades temporales. Es conveniente utilizar los tipos de datos de Pandas para los valores temporales de nuestras Series y DataFrames.

El uso de los diferentes valores e índices de tipo temporal de Pandas, nos permite utilizar funcionalidad avanzada como indexado parcial o de _slice_, desplazamientos de los puntos temporales, cambiar la frecuencia de un índice, o corregir por zona horaria.

No vamos a ver todos los detalles, pero se pueden encontrar en:

https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-overview

Los principales tipos de datos temporales soportados por Pandas son:

- `Timestamp` (`DatetimeIndex`), cuyo dtype es `datetime64[ns]`, y representa un instante concreto.
- `Period` (`PeriodIndex`), cuyo dtype es `period[freq]`, y representa un período regular.
- `Timedelta` (`TimedeltaIndex`), cuyo dtype es `timedelta64[ns]`, y representa una diferencia de tiempos.
- `DateOffset`, que es como `Timedelta` pero no es absoluto sino que respeta peculiaridades del calendario.

### Timestamp
Un `Timestamp` representa un instante en el tiempo. Una serie de Timestamps compone un `DateTimeIndex`, que puede usarse como índice de un DataFrame.

In [ ]:
ts = pd.Timestamp("2012-05-01 20:00")
ts

In [ ]:
print(ts.year, ts.week, ts.hour)

In [ ]:
pd.Timestamp(2012, 5, 1)

<p/>

`Timestamp` reconoce muchos formatos comunes.

In [ ]:
pd.Timestamp('03/01 22 16:15')

Para otros formatos, se debe usar el método `pd.to_datetime` con el argumento `format`.

<br/>

El método `pd.to_datetime` también nos permite convertir a `Timestamp` series de valores ya existentes.

In [ ]:
df = pd.DataFrame({
    'name':[ 'john','mary','peter','jeff','bill' ],
    'date_of_birth':[ '2000-01-01', '1999-12-20', '2000-11-01', '1995-02-25', '1992-06-30' ]
})
print(df.index)
df

In [ ]:
df.date_of_birth = pd.to_datetime(df.date_of_birth)
df.dtypes

In [ ]:
df.set_index('date_of_birth', inplace=True)
print(df.index)
df

<br/>

Podemos generar un `DateTimeIndex` con cierta frecuencia, usando la función `pd.date_range`.

In [ ]:
pd.date_range("2012-10-08", periods=4, freq="D")

In [ ]:
# Otro ejemplo, indicando momento inicial y final, e intervalo entre timestamps
idx = pd.date_range("2000-10-01 16:45:00", "2000-10-01 18:00:00", freq="10min", tz="UTC")
dft = pd.DataFrame({'num': np.arange(len(idx))}, index=idx)
dft

<br/>

Una vez tenemos definido un DateTimeIndex, podemos realizar indexado individual o _slice_ con _matching_ parcial, usando p. ej. strings (completos o parciales).

In [ ]:
dft.loc['2000-10-01 17:05:00']

In [ ]:
dft.loc['2000-10-01 17']

In [ ]:
dft.loc['2000-10-01 17:05':'2000-10-01 17:35']

### Period
Un `Period` representa un _intervalo_ (regular) de tiempo a partir de un instante determinado. Una serie de Periods constituye un `PeriodIndex`.

In [ ]:
pd.Period("2012-1-1", freq="D")

In [ ]:
periods = pd.period_range("1/1/2011", "7/1/2011", freq="M")
periods

In [ ]:
dfp = pd.DataFrame({'num': np.arange(len(periods))}, index=periods)
dfp

In [ ]:
dfp.loc['2011-02':'2011-06']

The function `to_period` allows you to convert from `TimeStamp` to `Period`.

In [ ]:
idx

In [ ]:
idx.to_period()

### Timedelta
Un `Timedelta` representa una _diferencia_ de tiempos relativa (no corresponde a un momento concreto). Un `DateOffset` es similar, pero respeta singularidades del calendario (por ejemplo si un día tiene una hora más o menos).

In [ ]:
ts2 = pd.Timestamp("2012-05-01 20:00")
ts1 = pd.Timestamp("2012-05-02 15:00")
ts2 - ts1

In [ ]:
ts1 + pd.offsets.Minute(5)

Podemos usar series de Timedelta en índices `TimedeltaIndex`.

In [ ]:
pd.TimedeltaIndex(["0 days", "10 days", "20 days"], freq="infer")

Podemos usar las funciones `to_timedelta` y `timedelta_range` de manera análogas a las funciones vistas anteriormente, y también indexado avanzado.

In [ ]:
pd.timedelta_range(start="1 days", periods=5)

<div style="background-color:powderblue;">

**EJERCICIO e8_5:** 

Vamos a explorar unos datos de la NASA sobre la actividad solar. En particular los siguientes:

- _sun_spots_: the sunspot number (R) - the number of spots on the surface of the Sun, indicating how active it is.
- _magnetic_activity_: the Dst index - an hourly magnetic activity index measured at Earth’s surface, in nT
- _radio_flux_: the F10.7 index - the radio flux at 10.7cm (i.e. how bright the Sun is at that wavelength), in “solar flux units” (sfu)

Para ello, haremos:    
    
- Cargar los datos del fichero `data/nasa_omni2.pickle` en un DataFrame

- Construir un índice de tipo fecha con las siguientes instrucciones:
```python
temp = df["Year"] * 100000 + df["DOY"] * 100 + df["Hour"]
df.index = pd.to_datetime(temp, format="%Y%j%H")
```


- Crear un plot con `df.plot`, usando `subplots=True` de las 3 series de datos, para todos los años.

- Repetir lo mismo pero solo para los datos entre los años 1987 y 1993.

- Utilizar la expresión `df.resample("1y").median()` para obtener la mediana por año (lo que elimina las frecuencias altas), y volver a realizar la gráfica para todos los años.   